In [ ]:
# default_exp core

# Hello nbdev

> API details.

In [ ]:
%autosave 5

Autosaving every 5 seconds


In [ ]:
# exports
from nbdev.showdoc import *
from fastcore.test import *
from typing import List
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

## Functions

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return(f'Hello {to}')

We can use this to say hello to someone.

In [ ]:
say_hello("Sylvain")

'Hello Sylvain'

In [ ]:
#test_eq(say_hello("Jeremy"), "Hello Jeremy!")

In [ ]:
#export
class HelloSayer:
    "Say hello to `to` using `say_hello`"
    def __init__(self, to): self.to = to
    def say(self):
        "do the saying"
        return say_hello(self.to)

In [ ]:
show_doc(HelloSayer.say)

<h4 id="HelloSayer.say" class="doc_header"><code>HelloSayer.say</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>HelloSayer.say</code>()

do the saying

In [ ]:
o = HelloSayer("Bichec")
o.say()

'Hello Bichec'

## USE Vectorizer

In [ ]:
# export
class UseVectorizer:
    """Wrapper for using Googles's Universal sentence encoder from Tensorflow Hub. 
    
    We have two types of encoders:
        * the regular USE
        * a dual encoder version for Q&A with a specific encoder for questions and responses
    
    **Args:**
        
        model_name(str): Name of the model to be loaded
    
    **Attributes:**
    
         MODELS(dict): Constant, that stores mapping between the model name and the TF Hub download URL
         
         model: Model loaded from TF Hub
    """

    MODELS = {
        "universal-sentence-encoder": "https://tfhub.dev/google/universal-sentence-encoder/4",
        "universal-sentence-encoder-large": "https://tfhub.dev/google/universal-sentence-encoder-large/5",
        "universal-sentence-encoder-multilingual": "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3",
        "universal-sentence-encoder-multilingual-large": "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3",
        "universal-sentence-encoder-qa": "https://tfhub.dev/google/universal-sentence-encoder-qa/3",
        "universal-sentence-encoder-multilingual-qa": "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3",
    }

    def __init__(self, model_name: str = "universal-sentence-encoder") -> None:
        self.model = hub.load(self.MODELS[model_name])

    def encode(
        self, texts: List, encoder_type: str = "question", response_contexts: List = []
    ) -> tf.Tensor:
        """Encodes an input text.
        
        **Args**:
            
            texts(List): List of strings to encode
            
            encoder_type(str): Only for Q&A models; 
                               choose question or response encoder.
            
            response_contexts: Only for the response encoder of a Q&A models;
                               usually the text around the answer text, for example it could be 
                               2 sentneces before plus 2 sentences after, it could also be 
                               the paragraph conaining the answer text. If no context is included, 
                               the response field is duplicated.
        **Returns**:
            2D Tensor containing vector representations of the input
        """
        # handle non Q&A models
        if "question_encoder" not in self.model.signatures:
            return self.model(texts)
        # handle Q&A question encoder
        if encoder_type == "question":
            texts = tf.constant(texts)
            return self.model.signatures["question_encoder"](texts)["outputs"]
        # handle Q&A response encoder
        if encoder_type == "response":
            texts = tf.constant(texts)
            if response_contexts:
                response_contexts = tf.constant(response_contexts)
            # duplicate response in case there is no context as suggested in the model docs
            else:
                response_contexts = texts
            return self.model.signatures["response_encoder"](
                input=texts, context=response_contexts
            )["outputs"]

In [ ]:
show_doc(UseVectorizer.encode)

<h4 id="UseVectorizer.encode" class="doc_header"><code>UseVectorizer.encode</code><a href="__main__.py#L32" class="source_link" style="float:right">[source]</a></h4>

> <code>UseVectorizer.encode</code>(**`texts`**:`List`\[`T`\], **`encoder_type`**:`str`=*`'question'`*, **`response_contexts`**:`List`\[`T`\]=*`[]`*)

Encodes an input text.

**Args**:
    
    texts(List): List of strings to encode
    
    encoder_type(str): Only for Q&A models; 
                       choose question or response encoder.
    
    response_contexts: Only for the response encoder of a Q&A models;
                       usually the text around the answer text, for example it could be 
                       2 sentneces before plus 2 sentences after, it could also be 
                       the paragraph conaining the answer text. If no context is included, 
                       the response field is duplicated.
**Returns**:
    2D Tensor containing vector representations of the input

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
